In [131]:
import msprime
import numpy as np
import random
from IPython.display import IFrame



# Generate a random tree sequence with record_full_arg=True so that you get marked recombination nodes
rs = random.randint(0,10000)   
ts = msprime.sim_ancestry(
    samples=3,
    recombination_rate=1e-8,
    sequence_length=2000,
    population_size=10_000,
    record_full_arg=True,
    random_seed=rs
)
print(rs)
print(ts.draw_text())



# Parameters for the dimensions of the D3 plot. Eventually want to handle this entirely in JS
width = 700 - 200
height = 600 - 100
w_spacing = width / (ts.num_samples - 1)
h_spacing = height / (ts.num_nodes - ts.num_samples - np.count_nonzero(ts.tables.nodes.flags == 131072)/2)



# Ordering of sample nodes is the same as the first tree in the sequence
ordered_nodes = []
for node in ts.first().nodes(order="minlex_postorder"):
    if node < ts.num_samples:
        ordered_nodes.append(node)

        
# Determines the rank (y position) of each time point       
unique_times = list(np.unique(ts.tables.nodes.time))



# The two recombination node IDs will be merged together in this visualization
recombination_nodes_to_merge = list(np.where(ts.tables.nodes.flags == 131072)[0])[1::2]



# Builds the nodes json. A "reference" is the id of another node that is used to determine a property in the
# graph. Example: recombination nodes should have the same x position as their child, unless their child is
# also a recombination node. This isn't yet implemented automatically in the layout as it breaks the force
# layout.
nodes = []
for ID, node in enumerate(ts.tables.nodes):
    info = {
        "id": ID,
        "flag": node.flags,
        "time": node.time,
        "fy": height-(unique_times.index(node.time)*h_spacing)+50 #fixed y position, property of force layout
    }
    label = ID
    if node.flags == 1:
        info["fx"] = ordered_nodes.index(ID)*w_spacing+100 #sample nodes have a fixed x position
    elif node.flags == 131072:
        if ID in recombination_nodes_to_merge:
            continue
        label = str(ID)+"/"+str(ID+1)
        info["x_pos_reference"] = ts.tables.edges[np.where(ts.tables.edges.parent == ID)[0]].child[0]
    elif node.flags == 262144:
        info["x_pos_reference"] = ts.tables.edges[np.where(ts.tables.edges.parent == ID)[0]].child[0]
    info["label"] = label #label which is either the node ID or two node IDs for recombination nodes
    nodes.append(info)
    
    
    
# Builds the edges json. For recombination nodes, replaces the larger number with the smaller. The direction
# that the edge should go relates to the positions of not just the nodes connected by that edge, but also the
# other edges connected to the child. See the JS for all of the different scenarios; still working through
# that.
links = []
for edge in ts.tables.edges:
    child = edge.child
    alternative_parent = ""
    if edge.parent not in recombination_nodes_to_merge:
        if ts.tables.nodes.flags[edge.child] == 131072:
            if edge.child in recombination_nodes_to_merge:
                alt_id = edge.child - 1
            else:
                alt_id = edge.child + 1
            alternative_parent = ts.tables.edges[np.where(ts.tables.edges.child == alt_id)[0]].parent[0]
        if edge.child in recombination_nodes_to_merge:
            child = edge.child - 1
        links.append({
            "source": edge.parent,
            "target": child,
            "direction_reference": alternative_parent #recombination nodes have an alternative parent
        })

            
        
with open("data.js", "w") as outfile:
    outfile.write("var graph = {\n")
    outfile.write("\tnodes: " + str(nodes) + ",\n")
    outfile.write("\tlinks: " + str(links) + "\n")
    outfile.write("};")
    
display(IFrame(src='./visualizer.html', width=750, height=650))

6300
59063.32┊       22    ┊       22    ┊       22    ┊       22    ┊             ┊  
        ┊     ┏━━┻━━┓ ┊     ┏━━┻━━┓ ┊     ┏━━┻━━┓ ┊     ┏━━┻━━┓ ┊             ┊  
40063.93┊    21     ┃ ┊    21     ┃ ┊    21     ┃ ┊    21     ┃ ┊       21    ┊  
        ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┏━━┻━━┓ ┊  
35478.66┊     ┃    19 ┊     ┃    19 ┊     ┃    19 ┊     ┃    19 ┊     ┃    20 ┊  
        ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊  
30500.64┊     ┃    18 ┊     ┃    18 ┊     ┃    18 ┊     ┃    18 ┊     ┃    18 ┊  
        ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊  
22923.15┊     ┃     ┃ ┊     ┃    17 ┊     ┃    17 ┊     ┃    17 ┊     ┃    17 ┊  
        ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊  
20830.58┊     ┃     ┃ ┊     ┃    15 ┊     ┃    15 ┊     ┃    16 ┊     ┃    16 ┊  
        ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊     ┃     ┃ ┊  
19845.61┊  